In [3]:
from huggingface_hub import notebook_login

notebook_login()


In [4]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [5]:
billsum = billsum.train_test_split(test_size=0.2)

In [6]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [17]:
import evaluate

rouge = evaluate.load("rouge")
rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [18]:
# import numpy as np


# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     result["gen_len"] = np.mean(prediction_lens)

#     return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [19]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [20]:
from transformers import pipeline

summarizer = pipeline("summarization", model="stevhliu/my_awesome_billsum_model")
summarizer(text)

Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': "the Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country."}]

In [85]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_billsum_model")
inputs = tokenizer(text, return_tensors="pt").input_ids
inputs.tolist()

[[21603,
  10,
  37,
  86,
  89,
  6105,
  419,
  8291,
  1983,
  1364,
  7,
  7744,
  2672,
  1358,
  6,
  533,
  124,
  1358,
  6,
  11,
  827,
  1358,
  5,
  94,
  31,
  7,
  8,
  167,
  8299,
  1041,
  30,
  3,
  26074,
  8,
  3298,
  5362,
  16,
  797,
  892,
  6,
  84,
  56,
  5656,
  95,
  797,
  2765,
  11,
  482,
  207,
  18,
  8832,
  53,
  6,
  7021,
  2476,
  640,
  8,
  684,
  5,
  94,
  31,
  195,
  1364,
  8,
  11724,
  11,
  987,
  8,
  6173,
  18,
  1123,
  138,
  189,
  63,
  11,
  11711,
  12,
  726,
  70,
  2725,
  698,
  5,
  275,
  150,
  80,
  492,
  365,
  1514,
  31471,
  399,
  215,
  56,
  726,
  3,
  9,
  23925,
  72,
  16,
  5161,
  5,
  1]]

In [69]:
import numpy as np
from qdrant_client.models import PointStruct

vectors = np.random.rand(100, 100)
client.upsert(
    collection_name="my_collection",
    points=[
        PointStruct(
            id=idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
        )
        for idx, vector in enumerate(vectors)
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [61]:
# from llama_index import GPTQdrantIndex

import qdrant_client

client = qdrant_client.QdrantClient(
    "localhost:6333",
    api_key=None, # For Qdrant Cloud, None for local instance
)

from qdrant_client.models import Distance, VectorParams

client.recreate_collection(
    collection_name="my_collection",
    vectors_config=VectorParams(size=100, distance=Distance.COSINE),
)
# index = GPTQdrantIndex.from_documents(documents, client=client, collection_name="documents")

True

In [91]:
import numpy as np
from qdrant_client.models import PointStruct

client.upsert(
    collection_name="my_collection",
    points=PointStruct(
            id=1000,
            vector=[ i * 1.0 for i in inputs.tolist()[0] ],
            payload={"color": "red", "rand_number": 111 % 10}
        )
    
)

UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Format error in JSON body: unknown variant `id`, expected `batch` or `points` at line 1 column 5"},"time":0.0}'

In [51]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("stevhliu/my_awesome_billsum_model")
outputs = model.generate(inputs, max_new_tokens=200, do_sample=True)
outputs

tensor([[    0,     8,    16,    89,  6105,  4709,  2876,    92,   704,    72,
          1645,    21,     8,     3,    23,    52,     9,  4703,    12,  1428,
          7744,  4845,     3,     5,    34,  1428,     7,  7744,  2672,  1358,
             6,   533,   124,  1358,     6,  5347,  1358,     6,    11,   827,
          1358,     3,     5,     8,   973,   133,  3033,    95, 10211,  2765,
            11,   482,   207,    18,  8832,    53,  2476,   640,     8,   684,
             3,     5,     1]])

In [57]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'the inflation reduction bill also offers more opportunities for the iracial to reduce prescription drugs. it reduces prescription drug costs, health care costs, labor costs, and energy costs. the law would raise up american workers and create good-paying jobs across the country.'

In [1]:
! pip install torch einops transformers accelerate
! pip install accelerate
# ! pip install --upgrade torch
# ! pip install request=="2.27.1"

In [1]:
from transformers import LlamaModel, LlamaConfig

# Initializing a LLaMA llama-7b style configuration
configuration = LlamaConfig()

# Initializing a model from the llama-7b style configuration
model = LlamaModel(configuration)

# Accessing the model configuration
configuration = model.config


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import os
os.environ['CURL_CA_BUNDLE'] = ''

# model_id="impira/layoutlm-document-qa"
model_id="tiiuae/falcon-7b"
tokenizer=AutoTokenizer.from_pretrained(model_id)
model=AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)

/home/joramas/miniconda/envs/thesis/lib/python3.9/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/joramas/miniconda/envs/thesis/lib/python3.9/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/joramas/miniconda/envs/thesis/lib/python3.9/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/

In [ ]:
from transformers import pipeline

nlp = pipeline(
    "document-question-answering",
    model="impira/layoutlm-document-qa",
)

nlp(
    "https://templates.invoicehome.com/invoice-template-us-neat-750px.png",
    "What is the invoice number?"
)
# {'score': 0.9943977, 'answer': 'us-001', 'start': 15, 'end': 15}

nlp(
    "https://miro.medium.com/max/787/1*iECQRIiOGTmEFLdWkVIH2g.jpeg",
    "What is the purchase amount?"
)

ValueError: Could not load model impira/layoutlm-document-qa with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForDocumentQuestionAnswering'>, <class 'transformers.models.layoutlm.modeling_layoutlm.LayoutLMForQuestionAnswering'>).

In [ ]:


sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Max retries exceeded with url: /repos/80/bd/80bd1e4a99ab55602d887f91215879d76ac3e1b877b8f683477be2952b8cb778/1c4b989693576ce79edf5f8f1a2d199d66c1448d35a126e52ba09cb67ca5da39?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00001-of-00002.bin%3B+filename%3D%22pytorch_model-00001-of-00002.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1690739196&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MDczOTE5Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84MC9iZC84MGJkMWU0YTk5YWI1NTYwMmQ4ODdmOTEyMTU4NzlkNzZhYzNlMWI4NzdiOGY2ODM0NzdiZTI5NTJiOGNiNzc4LzFjNGI5ODk2OTM1NzZjZTc5ZWRmNWY4ZjFhMmQxOTlkNjZjMTQ0OGQzNWExMjZlNTJiYTA5Y2I2N2NhNWRhMzk~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=yomeOAoycyBIWKChMcdkPMkRNhytdCIu1m6GtSVCQvdcfmYSyXqq5i9kP5hf0PSgGfMtmXBSX

In [ ]:

sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


ValueError: Could not load model gpt2 with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>).